# KymoButler Data Processing

After using KymoButler (AI program that analyzes kymographs made by Max Jakobs: https://github.com/MaxJakobs/KymoButler) to batch analyze your kymographs, it should output an excel file for each kymograph analyzed that contains different data about said kymograph (track duration, track displacement, start2end velocity, frame2frame velocity). The following python scripts will process and compile this data, it will sort the different metrics by direction (anterograde, retrograde, and total), then compile the different metrics for each kymograph result onto a master excel file. For example, it will compile the anterograde start2end velocity for each kymograph result onto one new excel. It saves all these compiled results into a folder called kymoresults. 

# 1 Replicate, 1 Condition

Goes into a single directory, processes and compiles all excel files in said directory, outputs compiled results to new folder in directory called kymoresults. When selecting a directory, choose the one containing all your kymobutler results excel files.

In [2]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
import warnings

# ignore openpyxl warning
warnings.simplefilter("ignore")

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the input directory using a file dialog
input_directory = filedialog.askdirectory(title="Select Input Directory")

# Check if a directory was selected
if not input_directory:
    print("No directory selected. Exiting...")
    exit()

# Concatenate the directory path with the new folder name
results_folder_path = os.path.join(input_directory, "kymoresults")

# Create the new folder
os.makedirs(results_folder_path, exist_ok=True)


def process_data (direction, column_title):

    # Create a list to store NumPy arrays
    result_arrays = []

    # Iterate over files in the directory
    for file_name in os.listdir(input_directory):
        file_path = os.path.join(input_directory, file_name)
        
        # Check if the item in the directory is an Excel file
        if file_path.endswith('.xlsx') or file_path.endswith('.xls'):
            print(f'Processing file: {file_name}')
            
            # Read the Excel file into a pandas DataFrame
            df = pd.read_excel(file_path)

            # Convert 'track total displacement [um]' and 'Direction' to numeric values
            df[column_title] = pd.to_numeric(df[column_title], errors='coerce')
            df['Direction'] = pd.to_numeric(df['Direction'], errors='coerce')

            if direction == 'ANT':
                # Filter rows starting from the 2nd row based on conditions
                filtered_rows = df.iloc[0:].loc[(~df[column_title].isna()) & (df['Direction'] == 1)]
            elif direction == 'RET':
                filtered_rows = df.iloc[0:].loc[(~df[column_title].isna()) & (df['Direction'] == -1)]
            elif direction == 'STAT':
                filtered_rows = df.iloc[0:].loc[(~df[column_title].isna()) & (df['Direction'] == 0)]
            else:
                filtered_rows = df.iloc[0:].loc[(~df[column_title].isna())]

            # Extract values from column 'track total displacement [um]' and convert them to a NumPy array
            result_array = np.array(filtered_rows[column_title])

            # Add the NumPy array to the list
            result_arrays.append(result_array)

    if result_arrays:
        # Create a DataFrame from the list of arrays
        result_df = pd.DataFrame(result_arrays).T
        result_df.columns = [f'Column_{i+1}' for i in range(result_df.shape[1])]

        if 'frame2frame' in column_title:
            # Save the resulting DataFrame to a CSV file
            result_df.to_csv(os.path.join(input_directory, 'kymoresults', f"{direction}frame2frameresults.csv"), index=False)
            print("Results saved successfully.")
        elif 'Start2end' in column_title: 
            result_df.to_csv(os.path.join(input_directory, 'kymoresults', f"{direction}start2endresults.csv"), index=False)
            print("Results saved successfully.")
        elif 'duration' in column_title: 
            result_df.to_csv(os.path.join(input_directory, 'kymoresults', f"{direction}durationresults.csv"), index=False)
            print("Results saved successfully.")
        else:
            result_df.to_csv(os.path.join(input_directory, 'kymoresults', f"{direction}displacementresults.csv"), index=False)
            print("Results saved successfully.")
    else:
        print("No valid data found to save.")

process_data( 'ANT', 'Av frame2frame velocity [um/sec]')
process_data('RET', 'Av frame2frame velocity [um/sec]')
process_data('STAT', 'Av frame2frame velocity [um/sec]')
process_data('TOTAL', 'Av frame2frame velocity [um/sec]')
process_data('ANT', 'Start2end velocity [um/sec]')
process_data('RET', 'Start2end velocity [um/sec]')
process_data('STAT', 'Start2end velocity [um/sec]')
process_data('TOTAL', 'Start2end velocity [um/sec]')
process_data('ANT', 'track duration [sec]')
process_data('RET', 'track duration [sec]')
process_data('STAT', 'track duration [sec]')
process_data('TOTAL', 'track duration [sec]')
process_data('ANT', 'track total displacement [um]')
process_data('RET', 'track total displacement [um]')
process_data('STAT', 'track total displacement [um]')
process_data('TOTAL', 'track total displacement [um]')


#Direction analysis

# Function to count occurrences of values in 'Direction' column
def count_directions(df):
    anterograde = (df['Direction'] == 1).sum()
    retrograde = (df['Direction'] == -1).sum()
    stationary = (df['Direction'] == 0).sum()
    total = len(df)
    return anterograde, retrograde, stationary, total

# Function to calculate percentages
def calculate_percentages(antero, retro, stationary, total):
    percent_antro = (antero / total) * 100
    percent_retro = (retro / total) * 100
    percent_stationary = (stationary / total) * 100
    return percent_antro, percent_retro, percent_stationary

# Initialize empty DataFrame to store results
result_df = pd.DataFrame(columns=[' '])

# Define the values for the first column
first_column_values = [ 'anterograde', 'retrograde', 'stationary', 'total', 'percent anterograde', 'percent retrograde', 'percent stationary']

# Insert the values into the first column of the dataframe
result_df.insert(0, 'Direction', first_column_values)

print(result_df)

#Iterate over files in the directory
for filename in os.listdir(input_directory):
    if filename.startswith('kymograph') and (filename.endswith('.xlsx') or filename.endswith('.xls')):
        filepath = os.path.join(input_directory, filename)
        # Read Excel file into DataFrame
        df = pd.read_excel(filepath)
        # Count occurrences of directions
        antero, retro, stationary, total = count_directions(df)
        # Calculate percentages
        percent_antro, percent_retro, percent_stationary = calculate_percentages(antero, retro, stationary, total)
        # Add results to DataFrame
        result_df[filename] = [antero, retro, stationary, total, percent_antro, percent_retro, percent_stationary]

# Save DataFrame to Excel file
result_df.to_csv(os.path.join(input_directory, 'kymoresults', 'directionresults.csv'), index=False)

Processing file: kymograph1.xlsx
Processing file: kymograph2.xlsx
Processing file: kymograph20240509151028.xlsx
Processing file: kymograph20240509151135.xlsx
Processing file: kymograph3.xlsx
Processing file: kymograph4.xlsx
Processing file: kymograph5.xlsx
Processing file: kymograph6.xlsx
Processing file: kymograph7.xlsx
Processing file: kymograph8.xlsx
Processing file: kymograph9.xlsx
Results saved successfully.
Processing file: kymograph1.xlsx
Processing file: kymograph2.xlsx
Processing file: kymograph20240509151028.xlsx
Processing file: kymograph20240509151135.xlsx
Processing file: kymograph3.xlsx
Processing file: kymograph4.xlsx
Processing file: kymograph5.xlsx
Processing file: kymograph6.xlsx
Processing file: kymograph7.xlsx
Processing file: kymograph8.xlsx
Processing file: kymograph9.xlsx
Results saved successfully.
Processing file: kymograph1.xlsx
Processing file: kymograph2.xlsx
Processing file: kymograph20240509151028.xlsx
Processing file: kymograph20240509151135.xlsx
Processi

# 1 Replicate, All Conditions

Goes into a directory that contains several folders, loops through each folder and compiles/processes data from each excel file within each folder. Intended to be used when you have a single replicate of an experiment, with several different conditions of data (ex. wt cells vs mutant cells). Each condition of data will have a separate kymoresults folder with output files. When selecting directory, select the one that contains all the condition folders.

In [1]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
import warnings

# ignore openpyxl warning
warnings.simplefilter("ignore")

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the input directory using a file dialog
input_directory = filedialog.askdirectory(title="Select Input Directory")

# Check if a directory was selected
if not input_directory:
    print("No directory selected. Exiting...")
    exit()


def process_data (direction, column_title):

    # Create a list to store NumPy arrays
    result_arrays = []

    # Iterate over files in the directory
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        # Check if the item in the directory is an Excel file
        if file_path.endswith('.xlsx') or file_path.endswith('.xls'):
            print(f'Processing file: {file_name}')
            
            # Read the Excel file into a pandas DataFrame
            df = pd.read_excel(file_path)

            # Convert 'track total displacement [um]' and 'Direction' to numeric values
            df[column_title] = pd.to_numeric(df[column_title], errors='coerce')
            df['Direction'] = pd.to_numeric(df['Direction'], errors='coerce')

            if direction == 'ANT':
                # Filter rows starting from the 2nd row based on conditions
                filtered_rows = df.iloc[0:].loc[(~df[column_title].isna()) & (df['Direction'] == 1)]
            elif direction == 'RET':
                filtered_rows = df.iloc[0:].loc[(~df[column_title].isna()) & (df['Direction'] == -1)]
            elif direction == 'STAT':
                filtered_rows = df.iloc[0:].loc[(~df[column_title].isna()) & (df['Direction'] == 0)]
            else:
                filtered_rows = df.iloc[0:].loc[(~df[column_title].isna())]

            # Extract values from column 'track total displacement [um]' and convert them to a NumPy array
            result_array = np.array(filtered_rows[column_title])

            # Add the NumPy array to the list
            result_arrays.append(result_array)

    if result_arrays:
        # Create a DataFrame from the list of arrays
        result_df = pd.DataFrame(result_arrays).T
        result_df.columns = [f'Column_{i+1}' for i in range(result_df.shape[1])]

        if 'frame2frame' in column_title:
            # Save the resulting DataFrame to a CSV file
            result_df.to_csv(os.path.join(folder_path, 'kymoresults', f"{direction}frame2frameresults.csv"), index=False)
            print("Results saved successfully.")
        elif 'Start2end' in column_title: 
            result_df.to_csv(os.path.join(folder_path, 'kymoresults', f"{direction}start2endresults.csv"), index=False)
            print("Results saved successfully.")
        elif 'duration' in column_title: 
            result_df.to_csv(os.path.join(folder_path, 'kymoresults', f"{direction}durationresults.csv"), index=False)
            print("Results saved successfully.")
        else:
            result_df.to_csv(os.path.join(folder_path, 'kymoresults', f"{direction}displacementresults.csv"), index=False)
            print("Results saved successfully.")
    else:
        print("No valid data found to save.")


for folder_name in os.listdir(input_directory):
    folder_path = os.path.join(input_directory, folder_name)
    
    # Check if the item in the directory is a folder
    if os.path.isdir(folder_path):
        #print('file exists')

        # Concatenate the directory path with the new folder name
        results_folder_path = os.path.join(folder_path, "kymoresults")

        # Create the new folder
        os.makedirs(results_folder_path, exist_ok=True)

        process_data( 'ANT', 'Av frame2frame velocity [um/sec]')
        process_data('RET', 'Av frame2frame velocity [um/sec]')
        process_data('STAT', 'Av frame2frame velocity [um/sec]')
        process_data('TOTAL', 'Av frame2frame velocity [um/sec]')
        process_data('ANT', 'Start2end velocity [um/sec]')
        process_data('RET', 'Start2end velocity [um/sec]')
        process_data('STAT', 'Start2end velocity [um/sec]')
        process_data('TOTAL', 'Start2end velocity [um/sec]')
        process_data('ANT', 'track duration [sec]')
        process_data('RET', 'track duration [sec]')
        process_data('STAT', 'track duration [sec]')
        process_data('TOTAL', 'track duration [sec]')
        process_data('ANT', 'track total displacement [um]')
        process_data('RET', 'track total displacement [um]')
        process_data('STAT', 'track total displacement [um]')
        process_data('TOTAL', 'track total displacement [um]')

        #Direction analysis

        # Function to count occurrences of values in 'Direction' column
        def count_directions(df):
            anterograde = (df['Direction'] == 1).sum()
            retrograde = (df['Direction'] == -1).sum()
            stationary = (df['Direction'] == 0).sum()
            total = len(df)
            return anterograde, retrograde, stationary, total

        # Function to calculate percentages
        def calculate_percentages(antero, retro, stationary, total):
            percent_antro = (antero / total) * 100
            percent_retro = (retro / total) * 100
            percent_stationary = (stationary / total) * 100
            return percent_antro, percent_retro, percent_stationary

        # Initialize empty DataFrame to store results
        result_df = pd.DataFrame(columns=[' '])

        # Define the values for the first column
        first_column_values = [ 'anterograde', 'retrograde', 'stationary', 'total', 'percent anterograde', 'percent retrograde', 'percent stationary']

        # Insert the values into the first column of the dataframe
        result_df.insert(0, 'Direction', first_column_values)

        #Iterate over files in the directory
        for filename in os.listdir(folder_path):
            if filename.startswith('kymograph') and (filename.endswith('.xlsx') or filename.endswith('.xls')):
                filepath = os.path.join(folder_path, filename)
                # Read Excel file into DataFrame
                df = pd.read_excel(filepath)
                # Count occurrences of directions
                antero, retro, stationary, total = count_directions(df)
                # Calculate percentages
                percent_antro, percent_retro, percent_stationary = calculate_percentages(antero, retro, stationary, total)
                # Add results to DataFrame
                result_df[filename] = [antero, retro, stationary, total, percent_antro, percent_retro, percent_stationary]

        # Save DataFrame to Excel file
        result_df.to_csv(os.path.join(folder_path, 'kymoresults', 'directionresults.csv'), index=False)
print('done')

Processing file: kymograph1.xlsx
Processing file: kymograph10.xlsx
Processing file: kymograph11.xlsx
Processing file: kymograph2.xlsx
Processing file: kymograph3.xlsx
Processing file: kymograph4.xlsx
Processing file: kymograph5.xlsx
Processing file: kymograph6.xlsx
Processing file: kymograph7.xlsx
Processing file: kymograph8.xlsx
Processing file: kymograph9.xlsx
Results saved successfully.
Processing file: kymograph1.xlsx
Processing file: kymograph10.xlsx
Processing file: kymograph11.xlsx
Processing file: kymograph2.xlsx
Processing file: kymograph3.xlsx
Processing file: kymograph4.xlsx
Processing file: kymograph5.xlsx
Processing file: kymograph6.xlsx
Processing file: kymograph7.xlsx
Processing file: kymograph8.xlsx
Processing file: kymograph9.xlsx
Results saved successfully.
Processing file: kymograph1.xlsx
Processing file: kymograph10.xlsx
Processing file: kymograph11.xlsx
Processing file: kymograph2.xlsx
Processing file: kymograph3.xlsx
Processing file: kymograph4.xlsx
Processing fil

# All Replicates, All Conditions

Intended to be used when you have several replicates of an experiment with several conditions per replicate (ex. 3 replicates of experiment with WT cells vs Mutant cells). Script will go into a directory that has several folders in it (one for each replicate of experiment), inside each folder is several more folders (one for each condition), then each condition folder has the kymobutler results excel files. It is important that each of the condition folders across replicates has the same name. When running the script, select the folder that contains all the replicate folders.

In [ ]:
import os
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
import warnings

# Ignore openpyxl warning
warnings.simplefilter("ignore")

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask user to select the input directory using a file dialog
input_directory = filedialog.askdirectory(title="Select Input Directory")

# Check if a directory was selected
if not input_directory:
    print("No directory selected. Exiting...")
    exit()

def process_data(folder_path, direction, column_title):
    # Create a list to store NumPy arrays
    result_arrays = []

    # Iterate over files in the directory
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        # Check if the item in the directory is an Excel file
        if file_path.endswith('.xlsx') or file_path.endswith('.xls'):
            print(f'Processing file: {file_name}')
            
            # Read the Excel file into a pandas DataFrame
            df = pd.read_excel(file_path)

            # Convert 'track total displacement [um]' and 'Direction' to numeric values
            df[column_title] = pd.to_numeric(df[column_title], errors='coerce')
            df['Direction'] = pd.to_numeric(df['Direction'], errors='coerce')

            if direction == 'ANT':
                # Filter rows starting from the 2nd row based on conditions
                filtered_rows = df.iloc[0:].loc[(~df[column_title].isna()) & (df['Direction'] == 1)]
            elif direction == 'RET':
                filtered_rows = df.iloc[0:].loc[(~df[column_title].isna()) & (df['Direction'] == -1)]
            elif direction == 'STAT':
                filtered_rows = df.iloc[0:].loc[(~df[column_title].isna()) & (df['Direction'] == 0)]
            else:
                filtered_rows = df.iloc[0:].loc[(~df[column_title].isna())]

            # Extract values from column 'track total displacement [um]' and convert them to a NumPy array
            result_array = np.array(filtered_rows[column_title])

            # Add the NumPy array to the list
            result_arrays.append(result_array)

    if result_arrays:
        # Create a DataFrame from the list of arrays
        result_df = pd.DataFrame(result_arrays).T
        result_df.columns = [f'Column_{i+1}' for i in range(result_df.shape[1])]

        if 'frame2frame' in column_title:
            # Save the resulting DataFrame to a CSV file
            result_df.to_csv(os.path.join(folder_path, 'kymoresults', f"{direction}frame2frameresults.csv"), index=False)
            print("Results saved successfully.")
        elif 'Start2end' in column_title: 
            result_df.to_csv(os.path.join(folder_path, 'kymoresults', f"{direction}start2endresults.csv"), index=False)
            print("Results saved successfully.")
        elif 'duration' in column_title: 
            result_df.to_csv(os.path.join(folder_path, 'kymoresults', f"{direction}durationresults.csv"), index=False)
            print("Results saved successfully.")
        else:
            result_df.to_csv(os.path.join(folder_path, 'kymoresults', f"{direction}displacementresults.csv"), index=False)
            print("Results saved successfully.")
    else:
        print("No valid data found to save.")

# Iterate over replicate folders
for replicate_folder in os.listdir(input_directory):
    replicate_folder_path = os.path.join(input_directory, replicate_folder)
    
    # Check if the item in the directory is a folder
    if os.path.isdir(replicate_folder_path):
        print(f'Processing replicate folder: {replicate_folder}')
        
        # Iterate over condition folders in the replicate folder
        for condition_folder in os.listdir(replicate_folder_path):
            condition_folder_path = os.path.join(replicate_folder_path, condition_folder)
            
            # Check if the item in the directory is a folder
            if os.path.isdir(condition_folder_path):
                print(f'Processing condition folder: {condition_folder}')
                
                # Concatenate the directory path with the new folder name
                results_folder_path = os.path.join(condition_folder_path, "kymoresults")

                # Create the new folder
                os.makedirs(results_folder_path, exist_ok=True)

                # Process data for different directions and column titles
                process_data(condition_folder_path, 'ANT', 'Av frame2frame velocity [um/sec]')
                process_data(condition_folder_path, 'RET', 'Av frame2frame velocity [um/sec]')
                process_data(condition_folder_path, 'STAT', 'Av frame2frame velocity [um/sec]')
                process_data(condition_folder_path, 'TOTAL', 'Av frame2frame velocity [um/sec]')
                process_data(condition_folder_path, 'ANT', 'Start2end velocity [um/sec]')
                process_data(condition_folder_path, 'RET', 'Start2end velocity [um/sec]')
                process_data(condition_folder_path, 'STAT', 'Start2end velocity [um/sec]')
                process_data(condition_folder_path, 'TOTAL', 'Start2end velocity [um/sec]')
                process_data(condition_folder_path, 'ANT', 'track duration [sec]')
                process_data(condition_folder_path, 'RET', 'track duration [sec]')
                process_data(condition_folder_path, 'STAT', 'track duration [sec]')
                process_data(condition_folder_path, 'TOTAL', 'track duration [sec]')
                process_data(condition_folder_path, 'ANT', 'track total displacement [um]')
                process_data(condition_folder_path, 'RET', 'track total displacement [um]')
                process_data(condition_folder_path, 'STAT', 'track total displacement [um]')
                process_data(condition_folder_path, 'TOTAL', 'track total displacement [um]')

                # Direction analysis

                # Function to count occurrences of values in 'Direction' column
                def count_directions(df):
                    anterograde = (df['Direction'] == 1).sum()
                    retrograde = (df['Direction'] == -1).sum()
                    stationary = (df['Direction'] == 0).sum()
                    total = len(df)
                    return anterograde, retrograde, stationary, total

                # Function to calculate percentages
                def calculate_percentages(antero, retro, stationary, total):
                    percent_antro = (antero / total) * 100
                    percent_retro = (retro / total) * 100
                    percent_stationary = (stationary / total) * 100
                    return percent_antro, percent_retro, percent_stationary

                # Initialize empty DataFrame to store results
                result_df = pd.DataFrame(columns=[' '])

                # Define the values for the first column
                first_column_values = [ 'anterograde', 'retrograde', 'stationary', 'total', 'percent anterograde', 'percent retrograde', 'percent stationary']

                # Insert the values into the first column of the dataframe
                result_df.insert(0, 'Direction', first_column_values)

                # Iterate over files in the directory
                for filename in os.listdir(condition_folder_path):
                    if filename.startswith('kymograph') and (filename.endswith('.xlsx') or filename.endswith('.xls')):
                        filepath = os.path.join(condition_folder_path, filename)
                        # Read Excel file into DataFrame
                        df = pd.read_excel(filepath)
                        # Count occurrences of directions
                        antero, retro, stationary, total = count_directions(df)
                        # Calculate percentages
                        percent_antro, percent_retro, percent_stationary = calculate_percentages(antero, retro, stationary, total)
                        # Add results to DataFrame
                        result_df[filename] = [antero, retro, stationary, total, percent_antro, percent_retro, percent_stationary]

                # Save DataFrame to Excel file
                result_df.to_csv(os.path.join(condition_folder_path, 'kymoresults', 'directionresults.csv'), index=False)

print('done')
